In [1]:
import itertools
from gurobipy import Model, GRB

def solve_linear_program():
    model = Model("Linear_Programming_Example")
    
    attributes = ["A", "B", "X", "Y", "C"]
    relations = [["A","B","X","Y", "C"], ["X","Y"], ["A","X"], ["A","Y"], ["B","X"], ["B", "Y"], ["C"]]
    variables = {}
    for i in range(1, len(attributes) + 1):
        for subset in itertools.combinations(attributes, i):
            name = "".join(subset)
            #name = "h(" + name + ")"
            # ub=len(subset)
            x = model.addVar(lb=0, ub=len(subset), name=name)
            variables[frozenset(subset)] = x
    
    model.setObjective(variables[frozenset(attributes)], GRB.MAXIMIZE)

    # Add constraints
    for rel in relations:
        model.addConstr(variables[frozenset(rel)] <= 1, name = "Rel_" + "".join(rel))
        #if len(rel) < 3:
        #    model.addConstr(variables[frozenset(rel)] <= len(rel), name = "Rel_" + "".join(rel))
     
    fds = [[["A", "B"], ["X", "Y", "C"]], 
           [["B", "X", "Y"], ["A", "C"]], 
           [["X", "C"], ["A", "B", "Y"]], 
           [["A", "X", "Y"], ["B", "C"]], 
           [["A", "C"], ["B", "X", "Y"]], 
           [["Y", "C"], ["A", "B", "X"]]]   
    #fds = [[["E","F","G","H"], ["A","B","C","D"]], [["A","B","C","E"], ["D","F","G","H"]], [["B","C","D","G"], ["A","E","F","H"]]]
    #candidate_keys = [[["E","F","G","H"], ["A","B","C","D", "E","F","G","H"]], [["A","B","C","E"], ["D","F","G","H", "A","B","C","E"]], [["B","C","D","G"], ["A","E","F","H", "B","C","D","G"]]]
    #fds = fds + candidate_keys
    #for Y, x in fds:
    #    Yx = Y + x
    #    model.addConstr(variables[frozenset(Yx)] == variables[frozenset(Y)], "fd_" + "".join(Y) + "->" + "".join(x))
        
        
    # non-Shannon
    if False:
        model.addConstr(11*variables[frozenset(attributes)] <=
                        3*variables[frozenset(["X", "Y"])] +
                        3*variables[frozenset(["A", "X"])] +
                        3*variables[frozenset(["A", "Y"])] +
                        variables[frozenset(["B", "X"])] + 
                        variables[frozenset(["B", "Y"])] +
                        5*variables[frozenset(["C"])] +
                        
                        variables[frozenset(attributes)] -
                        variables[frozenset(["A", "B"])] +
                        
                        4*variables[frozenset(attributes)] -
                        4*variables[frozenset(["A", "X", "Y"])] +
                        
                        variables[frozenset(attributes)] -
                        variables[frozenset(["B", "X", "Y"])] +
                        
                        variables[frozenset(attributes)] -
                        variables[frozenset(["A", "C"])] +
                        
                        variables[frozenset(attributes)] -
                        variables[frozenset(["X", "C"])] +
                        
                        variables[frozenset(attributes)] -
                        variables[frozenset(["Y", "C"])], name="Vamos")
        
    # Shannon inequalities
    if True:
        # Monotonicity
        for x in attributes:
            Y = [y for y in attributes if y != x]
            model.addConstr(variables[frozenset(attributes)] - variables[frozenset(Y)] >= 0, name="Monotonicity")
        
        # Submodularity
        for j, l in itertools.combinations(attributes, 2):
            S = [x for x in attributes if x != j and x != l]
            for i in range(1, len(S) + 1):
                for subset in itertools.combinations(S, i):
                    subset = list(subset)
                    jS = subset + [j]
                    lS = subset + [l]
                    jlS = subset + [j, l]
                    model.addConstr(variables[frozenset(jS)] + variables[frozenset(lS)] - variables[frozenset(subset)] - variables[frozenset(jlS)] >= 0, name="Submodularity")
        
        # Submodularity for those cases that subset S is empty (this reduces to subadditivity)
        for j, l in itertools.combinations(attributes, 2):
            model.addConstr(variables[frozenset([j])] + variables[frozenset([l])] - variables[frozenset([j, l])] >= 0, name="Subadditivity")
        
            
    # Save model to lp file
    model.write("Linear_Programming_Example2.lp")

    # Optimize the model
    model.optimize()

    # Check optimization status
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found!")
        print(f"Objective value = {model.objVal}")
    elif model.status == GRB.INFEASIBLE:
        print("The model is infeasible.")
    elif model.status == GRB.UNBOUNDED:
        print("The model is unbounded.")
    else:
        print(f"Optimization ended with status {model.status}")

# Call the function
solve_linear_program()


Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 92 rows, 31 columns and 327 nonzeros
Model fingerprint: 0xa71d089b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 0 columns
Presolve time: 0.00s
Academic license - for non-commercial use only - expires 2025-11-26
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 92 rows, 31 columns a